In [7]:
import numpy as np
import cv2
from skimage import io
from matplotlib import pyplot as plt
from google.colab.patches import cv2_imshow

def url_to_image(url):
  print("downloading %s" % (url))
  return cv2.cvtColor(io.imread(url), cv2.COLOR_BGR2RGB)

url = "https://raw.githubusercontent.com/mdavydov/ComputerVisionCourse/master/frames/v001.jpg"
img1 = url_to_image(url)

url = "https://raw.githubusercontent.com/mdavydov/ComputerVisionCourse/master/frames/v100.jpg"
img2 = url_to_image(url)

downloading https://raw.githubusercontent.com/mdavydov/ComputerVisionCourse/master/frames/v001.jpg
downloading https://raw.githubusercontent.com/mdavydov/ComputerVisionCourse/master/frames/v100.jpg


In [0]:
def init(A, B, patch_size):
  half = patch_size // 2
  B_h = B.shape[0]
  B_w = B.shape[1]
  A_h = A.shape[0]
  A_w = A.shape[1]
  
  x_offset = np.random.randint(0, B_h, A_h)
  y_offset = np.random.randint(0, B_w, A_w)
  
  F = np.zeros_like(A[:,:,0], dtype=object)
  for i in range(A_h):
    for j in range(A_w):
      F[i, j] = (x_offset[i], y_offset[j])
      
  return F

def dist(pix_a, pix_b, A, B, patch_size):
  half = patch_size // 2
  patch_A = A[pix_a[0]-half:pix_a[0]+half, pix_a[1]-half:pix_a[1]+half, :]
  patch_B = B[pix_b[0]-half:pix_b[0]+half, pix_b[1]-half:pix_b[1]+half, :]
  
  print(patch_A)
  print(patch_B)
  
  diff = patch_A - patch_B
  
  return np.sum(diff ** 2) / (patch_size ** 2)
  

def patch_match(A, B, patch_size, num_iter):
  F = init(A, B, patch_size)
  B_h = B.shape[0]
  B_w = B.shape[1]
  A_h = A.shape[0]
  A_w = A.shape[1]
  
  for it in range(1, num_iter + 1):
    if it % 2 == 0:
      for i in range(A_h-1, -1, -1):
        for j in range(A_w-1, -1, -1):
          print(F.shape)
          dist_right = dist((i, j + 1), F[i, j+1], A, B, patch_size)
          dist_down = dist((i+1, j), F[i+1, j], A, B, patch_size)
          dist_curr = dist((i, j), F[i, j], A, B, patch_size)
          
          argmin = np.argmin(np.array(dist_curr, dist_right, dist_down))
          
          if argmin == 1:
            F[i, j] = (i, j - 1)
          if argmin == 2:
            F[i, j] = (i - 1, j)
          
    else:
      for i in range(1, A_h):
        for j in range(1, A_w):
          print(A.shape)
          print(B.shape)
          #propagation
          dist_left = dist((i, j - 1), F[i,j-1], A, B, patch_size)
          dist_upper = dist((i - 1, j), F[i-1,j], A, B, patch_size)
          dist_curr = dist((i, j), F[i,j], A, B, patch_size)
          
          argmin = np.argmin(np.array(dist_curr, dist_left, dist_upper))
          
          if argmin == 1:
            F[i, j] = (i, j - 1)
          if argmin == 2:
            F[i, j] = (i - 1, j)
            
            
  return F
          

F = patch_match(img1, img2, 3, 5)
recon = np.zeros_like(img1)

for i in range(img1.shape[0]):
  for j in range(img1.shape[1]):
    recon[i, j, :] = img2[F[i, j][0], F[i, j][1], :]
    
cv2_imshow(recon)